# CV Lab project - Image Integration

Data Exploration

Description of the data:

* datasets:
    * train set, in each folder contains f-c.png and the json of the homography...
    * validation set, contains f-c.png, homographies and true label
    * test set, for submission

* 7 frames for each of the 10 cameras à 70 images for each sample 
    * f $\in$ {0,1,2,3,4,5,6}, where 3 is the center frame and
    * c $\in$ B01,...,B05,G01,...,G05 ,where B01 is the center camera

# ref:
* https://www.py4u.net/discuss/1950670

In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import re

# import torch
# import torchvision.transforms as T
# from torchvision.utils import draw_bounding_boxes
# from torchvision.models.vgg import vgg16
#
# model = vgg16(pretrained=True, progress=False)
# model = model.eval()


### Merging and Saving the Data

To merge and save merged the images in respective folders

In [2]:
def load_images_from_folder(folder, filter=None, masking=False):
    images = []
    mask = cv2.imread('data/mask.png',0)
    for filename in os.listdir(folder):
        img = None
        if filter is None:
            #print('?')
            img = cv2.imread(os.path.join(folder,filename))
            #apply masking
            if masking:
                img = cv2.bitwise_and(img,img,mask=mask)
        else:
            if filename in filter:
                #print(filename)
                img = cv2.imread(os.path.join(folder,filename))
                if masking:
                    img = cv2.bitwise_and(img,img,mask=mask)
                
        if img is not None:
            images.append(img)
    #print('Loaded {} images'.format(len(images)))
    return images

def makenumpy(df):
    m = None
    for r in df:
        if m is None:
            m = np.array(r).reshape(1,-1)
        else:
            m = np.concatenate((m,np.array(r).reshape(1,-1)),axis=0)
    return m

In [7]:
def merg_v1(warped):
    c = (warped > 0).sum(axis=0)
    warped_ = warped.sum(axis=0).squeeze()
    c[c==0] = 1.0
    c = 1.0/c
    return c * warped_

def merg_v2(warped):
    arr = np.zeros((warped[0].shape[0], warped[0].shape[1], 3))
    divide_arr = np.ones((warped[0].shape[0], warped[0].shape[1]))

    for l in range(len(warped)):
        (x, y, c) = (np.where(warped[l] != 0.0))
        divide_arr[x, y] += 1
        arr += warped[l]

    (x, y, c) = (np.where(arr == 0.0))
    divide_arr[x, y] = 1
    arr[:,:,0] /= divide_arr
    arr[:,:,1] /= divide_arr
    arr[:,:,2] /= divide_arr

    return arr

def merg_avg(warped):
    return np.array(warped).mean(axis=0)

def adjust_contrast_brightness(img, contrast:float=1.0, brightness:int=0):
    """
    Adjusts contrast and brightness of an uint8 image.
    contrast:   (0.0,  inf) with 1.0 leaving the contrast as is
    brightness: [-255, 255] with 0 leaving the brightness as is
    """
    brightness += int(round(255*(1-contrast)/2))
    return cv2.addWeighted(img, contrast, img, 0, brightness)/255


In [8]:
cwd = os.getcwd()
out_dir = os.path.join(cwd, "data_preprocessed")

dirs = ["train", "validation", "test"]
fns = ["train", "valid", "test"]
methods = ["merged", "bri_cont", "invert"]

for f in methods:
    for sub_f in fns:
        os.makedirs(os.path.join(out_dir, f, sub_f), exist_ok=True)

In [10]:
time_frames = 7
mask = cv2.imread('data/mask.png',0)
# jitter = T.ColorJitter(brightness=0.1, contrast=100)

for i, sdir in enumerate(dirs):
    train_folder_pattern = re.compile(fns[i] + '-(.*)-(.*)')
    for folder in os.listdir(os.path.join('data', sdir)):
        if train_folder_pattern.match(folder) is not None:
            print('prc Folder:',folder, end='')         
            homo = pd.read_json('data/' + sdir + '/' + folder + '/homographies.json', orient='record')
            for tf in range(time_frames):
                warped = []
                for fn in os.listdir('data/' + sdir + '/' + folder):
                    if ('merg' not in fn) and ('.png' in fn) and ( (str(tf)+'-') in fn):
                        print(' ', fn, end='')
                        img_f = './data/' + sdir + '/' + folder + '/' + fn
                        img = cv2.imread(img_f)

                        img = cv2.bitwise_and(img,img,mask=mask)
                        h = makenumpy( homo[fn.replace('.png', '')] )

                        warped += [cv2.warpPerspective(img/255., h, img.shape[:2] )]
                d = {"merg_avg_masked" : merg_avg(warped),
                     "merg_v1_masked" : merg_v1(np.array(warped)),
                     "merg_v2_masked": merg_v2(warped)}

                for k, v in d.items():
                    plt.imsave(os.path.join(out_dir, "merged", fns[i], f'{folder[:-3]}{str(tf)}-{k}.png'), v)

                    bri_cont = adjust_contrast_brightness(v, contrast=10, brightness=-120)
                    plt.imsave(os.path.join(out_dir, "bri_cont", fns[i], f'{folder[:-3]}{str(tf)}-{k}-bri-cont.png'), bri_cont)

                    inverted = ~bri_cont
                    plt.imsave(os.path.join(out_dir, "invert", fns[i], f'{folder[:-3]}{str(tf)}-{k}-invert.png'), inverted)

                # plt.imsave('data/'+dir+'/'+folder+'/'+str(tf)+'-merg_avg_masked.png',merged_avg)
                # plt.imsave('data/'+dir+'/'+folder+'/'+str(tf)+'-merg_v1_masked.png',merged_1)
                # plt.imsave('data/'+dir+'/'+folder+'/'+str(tf)+'-merg_v2_masked.png',merged_2)

                    
            print('\t')
            

prc Folder: train-1-0  0-B01.png  0-B02.png  0-B03.png  0-B04.png  0-B05.png  0-G01.png  0-G02.png  0-G03.png  0-G04.png  0-G05.png

ValueError: Floating point image RGB values must be in the 0..1 range.